In [1]:
from datasets import Dataset, load_dataset
import pandas as pd
from PIL import Image
import numpy as np
from io import BytesIO
import random

/Users/arnavakula/.pyenv/versions/3.12.4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("A2H0H0R1/plant-disease-new", split="train")

In [3]:
len(dataset)

162916

In [4]:
random_indices = random.sample(range(0, 162916), 10000)

In [5]:
len(random_indices)

10000

In [6]:
data = []

for i in random_indices:
    if dataset[i]['image'].size == (256, 256):
        img = np.array(dataset[i]['image'])
        label = dataset[i]['label']
        data.append({'image': img, 'label': label})
    else:
        print(dataset[i]['image'].size)

In [7]:
df = pd.DataFrame(data)

In [8]:
df.shape

(10000, 2)

In [9]:
len(df['label'].unique())

38

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Resizing, Rescaling
from tensorflow.keras.utils import to_categorical

In [11]:
images = np.stack(df['image'].values)

In [12]:
labels = df['label'].values

In [13]:
images = images / 255.0
labels = to_categorical(labels, num_classes=38)

In [20]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(38, activation='softmax')
])

/Users/arnavakula/.pyenv/versions/3.12.4/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
len(images), len(labels)

(10000, 10000)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

In [24]:
history = model.fit(images, labels, batch_size=64, epochs=10, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.1300 - loss: 3.6116 - val_accuracy: 0.3780 - val_loss: 2.4029
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.3853 - loss: 2.3022 - val_accuracy: 0.5510 - val_loss: 1.6735
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.5485 - loss: 1.5803 - val_accuracy: 0.6130 - val_loss: 1.3470
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.6742 - loss: 1.1051 - val_accuracy: 0.6520 - val_loss: 1.2012
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.7497 - loss: 0.8197 - val_accuracy: 0.6610 - val_loss: 1.1985
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.8071 - loss: 0.6225 - val_accuracy: 0.6790 - val_loss: 1.1258
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 174s 1s/step - accuracy: 0.8549 - loss: 0.4479 - val_accuracy: 0.7025 - val_loss: 1.1131
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.8869 - loss: 0.3657 - val_accu

In [26]:
model.save('../cnn-models/model.keras')